# Running serial simulations with OpenMM
In the penultimate example of this series, we show how easy managing, running, and chaining together OpenMM simulations can be with `polymerist`

In [1]:
import warnings 
warnings.catch_warnings(record=True)

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import is_empty, assemble_path


EXAMPLE_DIR = Path('prepared_system_examples')
assert EXAMPLE_DIR.exists() and not is_empty(EXAMPLE_DIR)

OUTPUT_DIR = Path('scratch_MD') # dummy directory for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

## Loading a prepared polymer topology
Check out the [previous demo all about this](3.1-MD_export_with_Interchange.ipynb) this if you haven't already

In [3]:
from openff.toolkit import ForceField
from openff.interchange import Interchange
warnings.filterwarnings('ignore', category=DeprecationWarning) # catches annoying DeprecationWarnings from Interchange

from polymerist.mdtools.openfftools.topology import topology_from_sdf
from polymerist.mdtools.openfftools.boxvectors import get_topology_bbox


prepared_polymer_system_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm_solv_water_TIP3P', 'sdf')
assert prepared_polymer_system_sdf.exists()
polymer_topology = topology_from_sdf(prepared_polymer_system_sdf)

forcefield = ForceField('openff-2.1.0.offxml') # choose a different forcefield here if you wish
interchange = Interchange.from_smirnoff(forcefield, polymer_topology, charge_from_molecules=list(polymer_topology.unique_molecules))
interchange.box = get_topology_bbox(polymer_topology)
interchange.visualize()

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
/home/timber/miniconda3/envs/polymerist-dev/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/timber/miniconda3/envs/polymerist-dev/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert vdW section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted vdW section from 0.3 to 0.4. `method="cutoff"` is now split into `periodic_method="cutoff"` and `nonperiodic_method="no-cutoff"`.
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert Electrostatics section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted Electrostatics section from 0.3 to 0.4. `method="PME"` is now split into `periodic_potential="Ewald3D-ConductingBoundary"`, `nonperiodic_potential="Coulomb"`, and `exception_potential="Coulomb"`.


NGLWidget()

## Defining simulation parameters
Also check out the dedicated demo on [defining simulation parameters](3.2-defining_simulation_parameters.ipynb) if you haven't already!

### Equilbration parameters: loaded from file for reproduction

In [4]:
from polymerist.mdtools.openmmtools.parameters import SimulationParameters


equil_params_path = assemble_path(EXAMPLE_DIR, 'equil_params', 'json')
assert equil_params_path.exists()

equilibration_params = SimulationParameters.from_file(equil_params_path)
print(equilibration_params.thermo_params)

ThermoParameters(ensemble='NPT', temperature=Quantity(value=300, unit=kelvin), pressure=Quantity(value=1, unit=atmosphere), friction_coeff=Quantity(value=1, unit=/picosecond), barostat_freq=25)


### Production parameters: made in-place to for portability
Note that this "production" is relatively short just so this demo doesn't take forever, but you can image making this as long as you need

In [5]:
from openmm.unit import femtosecond, picosecond, nanosecond, kelvin, atmosphere
from polymerist.mdtools.openmmtools.parameters import (
    ThermoParameters,
    IntegratorParameters,
    ReporterParameters,
    SimulationParameters
)

production_params =  SimulationParameters(
    integ_params=IntegratorParameters(
        time_step=2*femtosecond,
        total_time=0.2*nanosecond,
        num_samples=500,
    ),
    thermo_params=ThermoParameters(
        ensemble='NVT',
        temperature=300*kelvin,
        friction_coeff=1/picosecond,
    ),
    reporter_params=ReporterParameters(
        report_state=True,
        report_checkpoint=True,
        report_state_data=True,
    ),
)

## Running a simulation "schedule"
A "schedule" here denotes a serial sequence of simulations, each defined by their own SimulationParameters, each run in its own dedicated directory  
The initial topology, system, and positions provided will be passed to the first simulation in the schedule; all subsequent simulations will obtain these from the end of the previous simulation

<font color='red'>STATE IS NOT SHARED BETWEEN SUCCESSIVE SIMULATIONS IN A SCHEDULE</font>; At least as of this release, each simulation generate a new, empty State  
to avoid back-to-back simulations with potentially vastly-different thermodynamic parameters from contaminating one another

In [6]:
polymer_name : str = prepared_polymer_system_sdf.stem
POLYMER_SIM_DIR = OUTPUT_DIR / polymer_name
POLYMER_SIM_DIR.mkdir(exist_ok=True)

In [7]:
from polymerist.mdtools.openmmtools.forcegroups import impose_unique_force_groups
from openff.interchange.interop.openmm._positions import to_openmm_positions


# extract core OpenMM objects from Interchange
omm_topology = interchange.to_openmm_topology()
omm_system  = interchange.to_openmm(combine_nonbonded_forces=False)
omm_positions = to_openmm_positions(interchange, include_virtual_sites=True)
impose_unique_force_groups(omm_system) # ensure each Force is separate to enable mapping of energy contributions

In [9]:
from polymerist.genutils.logutils.IOHandlers import MSFHandlerFlex 
from polymerist.mdtools.openmmtools.execution import run_simulation_schedule


# MSFHandler compiles log and error events into single, "master" log file
logpath = assemble_path(POLYMER_SIM_DIR, 'simulation_status', 'log')
with MSFHandlerFlex(filename=logpath, proc_name=f'{polymer_name}_sims') as logger:
    history = run_simulation_schedule(
        POLYMER_SIM_DIR, 
        schedule={ # simulations will be run in the order they appear here
            'anneal'     : equilibration_params, 
            'production' : production_params,
        }, 
        init_top=omm_topology,
        init_sys=omm_system,
        init_pos=omm_positions,
        return_history=True
    )

INFO:polymerist.mdtools.openmmtools.execution:Initializing simulation 1/2 ("anneal")
INFO:polymerist.mdtools.openmmtools.thermo:Created MonteCarloBarostat Force(s) for NPT (Isothermal-isobaric) ensemble
INFO:polymerist.mdtools.openmmtools.preparation:Added MonteCarloBarostat Force to System
INFO:polymerist.mdtools.openmmtools.thermo:Created LangevinMiddleIntegrator for NPT (Isothermal-isobaric) ensemble
INFO:polymerist.mdtools.openmmtools.reporters:Prepared DCDReporter which reports to scratch_MD/PNIPAAm_solv_water_TIP3P/anneal/anneal_trajectory.dcd
INFO:polymerist.mdtools.openmmtools.reporters:Prepared CheckpointReporter which reports to scratch_MD/PNIPAAm_solv_water_TIP3P/anneal/anneal_checkpoint.chk
INFO:polymerist.mdtools.openmmtools.preparation:Setting positions in Context
INFO:polymerist.mdtools.openmmtools.execution:Performing energy minimization (initial PE = 458751377735.4615 kJ/mol)
INFO:polymerist.mdtools.openmmtools.execution:Energy successfully minimized (final PE = -10307